In [1]:
from ultralytics import YOLO 

In [2]:
import cv2
import os
import easyocr
import re

In [3]:
def extract_text_from_image(image_path):
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(['en'])

    # Read the image and extract text
    result = reader.readtext(image_path)

    # Extract all text
    extracted_text = []
    for detection in result:
        text = detection[1]
        extracted_text.append(text)

    return extracted_text

In [4]:
import csv

def create_csv_file():
    # Define the file path for the CSV file
    csv_file = 'vehicle_information.csv'

    # Create the CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Car ID','License Number', 'Car Type'])  # Write the header row

create_csv_file()

In [5]:
def save_vehicle_information_to_csv(vehicleID,vehicle_number, vehicle_type):
    # Define the file path for the CSV file
    csv_file = 'vehicle_information.csv'

    # Append the vehicle information to the CSV file
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([vehicleID, vehicle_number, vehicle_type])

In [6]:
class_names = {
    0: 'Hatchback',
    1: 'MidSize-SUV',
    3: 'SUV',
    4: 'Sedan'
}

In [7]:
model = YOLO('./best.pt')

In [8]:
# Load the image file
image_path = "./creta.jpg"
frame = cv2.imread(image_path)



# Run YOLOv8 inference on the image
results = model(frame)

cls = []
number_plate = ""
detected_car_type = ""

for result in results:
    boxes = result.boxes.cpu().numpy()
    for i, box in enumerate(boxes):
        if box.conf[0] > 0.4:
            cls = int(box.cls[0])
            if cls == 2:
                r = box.xyxy[0].astype(int)
                crop = frame[r[1]:r[3], r[0]:r[2]]
                cv2.imwrite("numberplate.jpg", crop)
                # Specify the path to the image
                number_plate = 'numberplate.jpg'
            elif cls != 2:
                car_type = class_names.get(cls)
                print(f"Car Type: {car_type}")
                detected_car_type = car_type

# Extract text from the image (implement your extract_text_from_image function)
text = extract_text_from_image(number_plate)
text = str(text)

# Define the pattern for number plate extraction using regular expressions
pattern = r'[A-Z]{2}\s?\d{2}\s?[A-Z]{2}\s?\d{4}'

# Search for the pattern in the extracted text
extracted_number_plates = re.findall(pattern, text)

# Initialize a list to store the results
car_results = []

# Iterate over the extracted number plates
for plate in extracted_number_plates:
    car_results.append((detected_car_type, plate))

# Print the extracted number plates along with the car type
i = 0   #Vehicle ID
for plate, car_type in car_results:
    i+=1
    save_vehicle_information_to_csv(i,plate,car_type)



Ultralytics YOLOv8.0.23  Python-3.11.0 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs
0: 192x256 1 MidSize-SUV, 1 NumberPlate, 81.8ms
Speed: 26.1ms pre-process, 81.8ms inference, 556.4ms postprocess per image at shape (1, 3, 256, 256)


Car Type: MidSize-SUV


In [7]:
import tkinter as tk
import pandas as pd

# CSV TO API

In [31]:
import tkinter as tk
import csv

def create_grid(frame, rows, columns):
    for i in range(rows):
        frame.rowconfigure(i, weight=1, minsize=50)
        for j in range(columns):
            frame.columnconfigure(j, weight=1, minsize=50)
            cell = tk.Frame(
                master=frame,
                relief=tk.RAISED,
                borderwidth=1
            )
            cell.grid(row=i, column=j, sticky="nsew")
            cell_colors.append(cell)  # Store the cell frame for color assignment

def assign_vehicle_color(vehicle_type):
    if vehicle_type == "SUV":
        return "red"
    elif vehicle_type == "Sedan":
        return "blue"
    elif vehicle_type == "MidSize-SUV":
        return "green"
    elif vehicle_type == "Hatchback":
        return "yellow"
    else:
        return "white"  # Default color if vehicle type is unknown or not provided

def read_vehicle_data(filename):
    vehicle_data = []
    with open(filename, "r") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            vehicle_id, vehicle_type, vehicle_number = row
            vehicle_data.append((vehicle_id, vehicle_type.strip(), vehicle_number))
    return vehicle_data

def populate_grids(vehicle_data):
    for i, (vehicle_id, vehicle_type, vehicle_number) in enumerate(vehicle_data):
        cell = cell_colors[i % len(cell_colors)]
        color = assign_vehicle_color(vehicle_type)
        
        # Configure cell's border and background color
        cell.config(bg=color, highlightbackground="black", highlightthickness=1.25)
        
        # Clear any existing widgets in the cell
        for widget in cell.winfo_children():
            widget.destroy()
        
        # Create label with custom font style and size
        label_font = ("Sans-Serif", 12, "bold")
        label = tk.Label(cell, text=f"Vehicle ID: {vehicle_id}\n\nType: {vehicle_type}\n\nLicense: {vehicle_number}", bg=color, fg="white", font=label_font)
        label.pack(fill=tk.BOTH, expand=True)

def create_window(vehicle_data):
    window = tk.Tk()
    window.title("Vehicle Data")
    
    # Calculate the number of grids needed
    num_grids = (len(vehicle_data) + 9) // 10

    for i in range(num_grids):
        # Create the main frame for each grid
        main_frame = tk.Frame(master=window)
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Create the grid
        create_grid(main_frame, 2, 5)

        # Populate the grids with vehicle data
        start_index = i * 10
        end_index = min((i + 1) * 10, len(vehicle_data))
        populate_grids(vehicle_data[start_index:end_index])

    window.mainloop()

# Read vehicle data from CSV file
filename = "vehicle_information.csv"  # Replace with your own filename
vehicle_data = read_vehicle_data(filename)

# List to store cell frames for color assignment
cell_colors = []

# Create the window
create_window(vehicle_data)


In [5]:
cv2.imshow()

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'winname' (pos 1)
>  - imshow() missing required argument 'winname' (pos 1)
>  - imshow() missing required argument 'winname' (pos 1)


In [ ]:
class_names = {
    0: 'Hatchback',
    1: 'MidSize-SUV',
    3: 'SUV',
    4: 'Sedan'
}

print(vehicle_)

In [29]:
print(vehicle_type)

MidSize-SUV


# VIDEO INPUTS TRIAL

In [14]:
import cv2
import easyocr
import re
import csv


In [15]:
reader = easyocr.Reader(['en'])

In [16]:
class_names = {
    0: 'Hatchback',
    1: 'MidSize-SUV',
    3: 'SUV',
    4: 'Sedan'
}

In [35]:
video_path = "./entry.mp4"
cap = cv2.VideoCapture(video_path)

In [36]:
vehicle_types = set()
vehicle_numbers = set()


In [37]:
csv_file = open("vehicle_information.csv", "a", newline="")
csv_writer = csv.writer(csv_file)


In [38]:
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Run YOLOv8 inference on the frame to get the results
    results = model(frame)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for i, box in enumerate(boxes):
            if box.conf[0] > 0.4:
                cls = int(box.cls[0])
                if cls == 2:
                    r = box.xyxy[0].astype(int)
                    crop = frame[r[1]:r[3], r[0]:r[2]]
                    cv2.imwrite("numberplate.jpg", crop)
                    # Specify the path to the image
                    number_plate = 'numberplate.jpg'
                elif cls != 2:
                    car_type = class_names.get(cls)
                    print(f"Car Type: {car_type}")
                    detected_car_type = car_type

        # Extract text from the image (implement your extract_text_from_image function)
        text = extract_text_from_image(number_plate)
        text = str(text)

        # Define the pattern for number plate extraction using regular expressions
        pattern = r'[A-Z]{2}\s?\d{2}\s?[A-Z]{2}\s?\d{4}'

        # Search for the pattern in the extracted text
        extracted_number_plates = re.findall(pattern, text)

        # Iterate over the extracted number plates
        for plate in extracted_number_plates:
            # Add the vehicle type and number to the respective sets
            vehicle_types.add(detected_car_type)
            vehicle_numbers.add(plate)


0: 160x256 1 NumberPlate, 1 Sedan, 18.0ms
Speed: 1.0ms pre-process, 18.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: Sedan


0: 160x256 1 NumberPlate, 1 Sedan, 9.0ms
Speed: 0.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: Sedan


0: 160x256 1 NumberPlate, 1 Sedan, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


...

Car Type: SUV


0: 160x256 1 NumberPlate, 1 SUV, 9.0ms
Speed: 1.0ms pre-process, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: SUV


0: 160x256 1 NumberPlate, 1 SUV, 8.0ms
Speed: 1.0ms pre-process, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 256)


Car Type: SUV


In [32]:
cap.release()
csv_file.close()


In [33]:
with open("vehicle_information.csv", "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for vehicle_type, vehicle_number in zip(vehicle_types, vehicle_numbers):
        csv_writer.writerow([vehicle_type, vehicle_number])


In [34]:
for plate, car_type in zip(vehicle_types, vehicle_numbers):
    print(f"This car with number: {plate} is a {car_type} type.")


In [36]:
import tkinter as tk
import csv

class ParkingLot:
    def __init__(self, capacity):
        self.capacity = capacity
        self.spaces = [None] * capacity

class Car:
    def __init__(self, car_type, car_number):
        self.car_type = car_type
        self.car_number = car_number

def allocate_parking_space(car_type, car_number, parking_lots):
    for parking_lot in parking_lots:
        if parking_lot.spaces.count(None) == 0:
            continue  # Move to the next parking lot if the current one is full

        for i, space in enumerate(parking_lot.spaces):
            if space is None:
                if car_type == 'SUV':
                    suv_count = sum(1 for s in parking_lot.spaces if s and s.car_type == 'SUV')
                    if suv_count >= 0.4 * parking_lot.capacity:
                        continue  # SUV limit reached in this parking lot

                if car_type == 'SUV' and any(s and s.car_type == 'SUV' for s in parking_lot.spaces[i - 1:i + 2]):
                    continue  # Don't allow two SUVs to be near

                if car_type in ['Sedan', 'Hatchback']:
                    neighboring_count = 0
                    for j in range(i - 1, i + 2):
                        if 0 <= j < parking_lot.capacity and parking_lot.spaces[j] and parking_lot.spaces[
                            j].car_type in ['Sedan', 'Hatchback']:
                            neighboring_count += 1
                    if neighboring_count >= 2:
                        continue  # Limit reached for Sedans and Hatchbacks

                parking_lot.spaces[i] = Car(car_type, car_number)
                return f'Car {car_number} parked at Lot {parking_lots.index(parking_lot) + 1}, Space {i + 1}'

    return 'No available parking space'

def read_vehicle_data(filename):
    vehicle_data = []
    with open(filename, "r") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            vehicle_id, vehicle_number, vehicle_type = row
            vehicle_data.append((vehicle_id, vehicle_number, vehicle_type.strip()))
    return vehicle_data

def create_grid(frame, rows, columns):
    for i in range(rows):
        frame.rowconfigure(i, weight=1, minsize=50)
        for j in range(columns):
            frame.columnconfigure(j, weight=1, minsize=50)
            cell = tk.Frame(
                master=frame,
                relief=tk.RAISED,
                borderwidth=1
            )
            cell.grid(row=i, column=j, sticky="nsew")
            cell_frames.append(cell)

def populate_grids(vehicle_data):
    for i, (vehicle_id, _, vehicle_type) in enumerate(vehicle_data):
        cell = cell_frames[i % len(cell_frames)]
        color = assign_vehicle_color(vehicle_type)
        cell.config(bg=color)

        # Clear any existing widgets in the cell
        for widget in cell.winfo_children():
            widget.destroy()

        label_font = ("Arial", 12, "bold")
        label = tk.Label(cell, text=f"Vehicle ID: {vehicle_id}\nType: {vehicle_type}", bg=color, fg="white", font=label_font)
        label.pack(fill=tk.BOTH, expand=True)

def assign_vehicle_color(vehicle_type):
    colors = {'SUV': 'red', 'Mid Size SUV': 'purple', 'Sedan': 'green', 'Hatchback': 'blue'}
    return colors.get(vehicle_type, 'white')

def load_vehicle_data():
    filename = "input.csv"  # Update with your CSV file path
    vehicle_data = read_vehicle_data(filename)
    populate_grids(vehicle_data)

# Initialize Tkinter window
window = tk.Tk()
window.title("Parking Lot")
window.geometry("800x600")

# Create main frame
main_frame = tk.Frame(master=window)
main_frame.pack(fill=tk.BOTH, expand=True)

# Create cell frames
cell_frames = []
create_grid(main_frame, 2, 5)

# Load vehicle data and populate grids
load_vehicle_data()

# Add a button to reload vehicle data
button_frame = tk.Frame(master=window)
button_frame.pack(pady=10)
reload_button = tk.Button(button_frame, text="Reload Vehicle Data", command=load_vehicle_data)
reload_button.pack()

# Start the Tkinter event loop
window.mainloop()


In [42]:
def populate_grids(vehicle_data):
    for i, (vehicle_id, _, vehicle_type) in enumerate(vehicle_data):
        allocated = False
        for cell in cell_frames:
            color = assign_vehicle_color(vehicle_type)
            if not cell.winfo_children():  # Check if cell is empty
                row, col = cell.grid_info()["row"], cell.grid_info()["column"]
                if can_allocate_vehicle(row, col, vehicle_type):
                    cell.config(bg=color)
                    label_font = ("Arial", 12, "bold")
                    label = tk.Label(cell, text=f"Vehicle ID: {vehicle_id}\nType: {vehicle_type}", bg=color, fg="white", font=label_font)
                    label.pack(fill=tk.BOTH, expand=True)
                    allocated = True
                    break

        if not allocated:
            # Create a new grid layout and allocate the vehicle
            create_grid(main_frame, 2, 5)
            cell = cell_frames[-1]
            color = assign_vehicle_color(vehicle_type)
            cell.config(bg=color)
            label_font = ("Arial", 12, "bold")
            label = tk.Label(cell, text=f"Vehicle ID: {vehicle_id}\nType: {vehicle_type}", bg=color, fg="white", font=label_font)
            label.pack(fill=tk.BOTH, expand=True)

def can_allocate_vehicle(row, col, vehicle_type):
    if vehicle_type == 'SUV':
        suv_count = sum(1 for cell in cell_frames if cell.winfo_children() and cell.winfo_children()[0]['text'].split("\n")[1] == 'SUV')
        if suv_count >= 0.4 * len(cell_frames):
            return False
        if row > 0 and cell_frames[(row - 1) * 5 + col].winfo_children() and cell_frames[(row - 1) * 5 + col].winfo_children()[0]['text'].split("\n")[1] == 'SUV':
            return False
        if row < 1 and cell_frames[(row + 1) * 5 + col].winfo_children() and cell_frames[(row + 1) * 5 + col].winfo_children()[0]['text'].split("\n")[1] == 'SUV':
            return False
        if col > 0 and cell_frames[row * 5 + col - 1].winfo_children() and cell_frames[row * 5 + col - 1].winfo_children()[0]['text'].split("\n")[1] == 'SUV':
            return False
        if col < 4 and cell_frames[row * 5 + col + 1].winfo_children() and cell_frames[row * 5 + col + 1].winfo_children()[0]['text'].split("\n")[1] == 'SUV':
            return False
    else:
        neighboring_count = 0
        if row > 0 and cell_frames[(row - 1) * 5 + col].winfo_children() and cell_frames[(row - 1) * 5 + col].winfo_children()[0]['text'].split("\n")[1] in ['Sedan', 'Hatchback']:
            neighboring_count += 1
        if row < 1 and cell_frames[(row + 1) * 5 + col].winfo_children() and cell_frames[(row + 1) * 5 + col].winfo_children()[0]['text'].split("\n")[1] in ['Sedan', 'Hatchback']:
            neighboring_count += 1
        if col > 0 and cell_frames[row * 5 + col - 1].winfo_children() and cell_frames[row * 5 + col - 1].winfo_children()[0]['text'].split("\n")[1] in ['Sedan', 'Hatchback']:
            neighboring_count += 1
        if col < 4 and cell_frames[row * 5 + col + 1].winfo_children() and cell_frames[row * 5 + col + 1].winfo_children()[0]['text'].split("\n")[1] in ['Sedan', 'Hatchback']:
            neighboring_count += 1
        if neighboring_count >= 2:
            return False
    return True

def load_vehicle_data():
    filename = "input.csv"  # Update with your CSV file path
    vehicle_data = read_vehicle_data(filename)
    populate_grids(vehicle_data)

# Initialize Tkinter window
window = tk.Tk()
window.title("Parking Lot")
window.geometry("800x600")

# Create main frame
main_frame = tk.Frame(master=window)
main_frame.pack(fill=tk.BOTH, expand=True)

# Create cell frames
cell_frames = []
create_grid(main_frame, 2, 5)

# Load vehicle data and populate grids
load_vehicle_data()

# Add a button to reload vehicle data
button_frame = tk.Frame(master=window)
button_frame.pack(pady=10)
reload_button = tk.Button(button_frame, text="Reload Vehicle Data", command=load_vehicle_data)
reload_button.pack()

# Start the Tkinter event loop
window.mainloop()
